In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena
import mz_unit_dur_freq as mz_udf

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

sns.despine()

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
        'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
        'axes.linewidth':2.5, 'lines.linewidth': 2.5,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

set_pub_plots()

In [ ]:
pal=sns.blend_palette(['cyan','magenta','grey','crimson'], 4)
sns.palplot(pal)
sns.set_palette(pal)

---

# First, load in the previously saved dataFrame

In [5]:
V1 = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\V1Hippo_spikes\v1_rg_df.pkl")
HPC = pd.read_pickle(r"C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\V1Hippo_spikes\hpc_rg_df.pkl")

# combine the pre/post df with the novel df
V1HPC_combo = pd.concat([V1, HPC])

In [ ]:
V1HPC_combo.head(2)

In [ ]:
V1HPC_combo.stim.unique()

# Choose which r_group to plot

In [12]:
stim_cond = 'post'

v1_rgroup = [0,1,2,3]
hpc_rgroup = [0]

# change the dataFrame depending on r_groups or r2_groups (only_v1_kmeans,only_hpc_kmeans or v1_oscillators_kmeans,hpc_oscillators_kmeans)
V1_rg = V1['r2_groups'].isin(v1_rgroup)
V1_rg = V1[V1_rg]
HPC_rg = HPC['r2_groups'].isin(hpc_rgroup)
HPC_rg = HPC[HPC_rg]
kmeans_all = pd.concat([V1_rg, HPC_rg])
kmeans_all = kmeans_all.sort_values(by=['r_groups'])

In [13]:
is_sorted = 'yes'

v1_km = V1_rg.pivot('cuid', 'times', 'zscore')
hip_km = HPC_rg.pivot('cuid', 'times', 'zscore')

if is_sorted == 'yes':
    v1_km_np = v1_km.values[ np.argsort(np.mean(v1_km.values[:,50:80], axis = 1) )] #if I want sorted activity
    hip_km_np = hip_km.values[ np.argsort(np.mean(hip_km.values[:,50:80], axis = 1) )] #if I want sorted activity
elif is_sorted == 'no':
    v1_km_np = v1_km.values
    hip_km_np = hip_km.values

In [14]:
sp_dims = (15, 4) #(width, height)
v_max = 5
v_min = -1
xvalues = np.arange(start=0.4, stop=1.7, step=0.2)
y_min = -2
y_max = 6

In [ ]:
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs'

f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=(15,4))
f.tight_layout()
plt.suptitle('kmeans: '+stim_cond)
plt.minorticks_on()

# Heatmap for A
sns.heatmap(v1_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel(str(len(V1_rg.cuid.unique())))
# ax1.set_xticklabels(xvalues, rotation=0)
ax1.set_xlim(0.4*100,1.6*100)
ax1.set_title('V1')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(hip_km_np, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel(str(len(HPC_rg.cuid.unique())))
# ax2.set_xticklabels(xvalues, rotation=0)
ax2.set_xlim(0.4*100,1.6*100)
ax2.set_title('HPC')
cbar = ax2.collections[0].colorbar
# cbar.ax.tick_params(labelsize=25)
cbar.set_label('Z-score')

# Change the end of this next line with a new file name!!
if to_plot == 1:
    fig_title = str('V1HPC_oscillators_heat.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)
    
plt.show()

In [ ]:
sp_dims = (10, 4) #(width, height)
f, ax = plt.subplots(1,1, sharex='col', figsize=sp_dims)

#Combo lineplot
sns.lineplot(x = 'times', y = 'ztc', 
             estimator = 'mean', 
             data = kmeans_all, 
             hue = 'region',
             hue_order = ['v1','hippo'],
             legend=False,
             ci=None) #ci is the error

plt.legend(['V1','HPC'], bbox_to_anchor=(1.1, 1.05))

plt.axvspan(0.5, 0.7, color='lightgrey', alpha = 0.5)
plt.axhline(y=0, xmin=0, xmax=3, c='black', linewidth=1, zorder=0)

plt.xlim(0.4,1.6)
plt.xticks([0.4,0.6,0.8,1,1.2,1.4,1.6])
# plt.yticks([-4,0,4,8,12,16,20])
# plt.ylim([-4,22])
plt.minorticks_on()
plt.xlabel('Time (s)')
plt.ylabel('zscore')
plt.title('kmeans: '+stim_cond)

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs'
if to_plot == 1:
    fig_title = str('V1HPC_oscillators_line.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)
    
plt.show()

---

# Plotting histogram of units with peaks

In [ ]:
group_A_df = kmeans_all[kmeans_all['region'] == 'v1']
group_B_df = kmeans_all[kmeans_all['region'] == 'hippo']

A_array = np.array(group_A_df.pivot(index='cuid', columns='times', values='zscore'))
print(A_array.shape) #this is [units x times] and the values are above

B_array = np.array(group_B_df.pivot(index='cuid', columns='times', values='zscore'))
print(B_array.shape) #this is [units x times] and the values are above

In [18]:
num_cycles_required = 1
is_sorted = 'yes'

Aunit_3plus = []
Aind_3plus = []
for unit in range(A_array.shape[0]):
    yy = A_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    if len(peak_ind) >= num_cycles_required:
        Aunit_3plus.append(yy)
        Aind_3plus.append(peak_ind)

Bunit_3plus = []
Bind_3plus = []
for unit in range(B_array.shape[0]):
    yy = B_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    if len(peak_ind) >= num_cycles_required:
        Bunit_3plus.append(yy)
        Bind_3plus.append(peak_ind)

Aunit_3plus = np.array(Aunit_3plus)
Bunit_3plus = np.array(Bunit_3plus)
Aind_3plus = np.concatenate(Aind_3plus, axis=0)
Bind_3plus = np.concatenate(Bind_3plus, axis=0)

In [19]:
if is_sorted == 'yes':
    v1_3plus = Aunit_3plus[np.argsort(np.mean(Aunit_3plus[:,50:60], axis = 1) )] #if I want sorted activity
    hip_3plus = Bunit_3plus[np.argsort(np.mean(Bunit_3plus[:,50:60], axis = 1) )] #if I want sorted activity
    
#     v1_3plus = v1_3plus[0:104,:]
#     hip_3plus = hip_3plus[9:13,:]
    
elif is_sorted == 'no':
    v1_3plus = Aunit_3plus
    hip_3plus = Bunit_3plus
stim_cond = 'post 3+'

In [20]:
sp_dims = (15, 4) #(width, height)
v_max = 5
v_min = -1
xvalues = np.arange(start=0.4, stop=1.7, step=0.2)
y_min = -2
y_max = 6

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2, sharex='col', figsize=sp_dims)
f.tight_layout()
plt.minorticks_on()

# Heatmap for A
sns.heatmap(v1_3plus, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax1)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel(str(v1_3plus.shape[0]))
ax1.set_xlim(0.4*100,1.6*100)
ax1.set_title('V1')
cbar = ax1.collections[0].colorbar
cbar.remove()

#Heatmap for B
sns.heatmap(hip_3plus, 
            cmap = 'afmhot', xticklabels=20, yticklabels=False, 
            vmax=v_max, center = 2, vmin = v_min, robust = True, ax=ax2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel(str(hip_3plus.shape[0]))
ax2.set_xlim(0.4*100,1.6*100)
ax2.set_title('Hippo')
cbar = ax2.collections[0].colorbar
cbar.ax.tick_params(labelsize=25)
cbar.set_label('Z-score')

# Change the end of this next line with a new file name!!
# plt.savefig(r"U:\Papers\D&P Osc in Mouse Vis Sys\J Neuro Submission\Figures\V1_Hippocampus\panels\more_post\post3_heat.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 9.0 # x dimension
fig_size[1] = 5.0 # y dimension

sns.histplot(Aind_3plus, binwidth=4, linewidth=0, stat='probability', color='cyan', label='V1', 
             kde=True, kde_kws={'bw_adjust' : 0.15})
sns.histplot(Bind_3plus, binwidth=4, linewidth=0, stat='probability', color='magenta', label='HPC', 
             kde=True, kde_kws={'bw_adjust' : 0.15})
# sns.kdeplot(Aind_3plus, bw_method=0.05, color='cyan', fill=True, label='V1')
# sns.kdeplot(Bind_3plus, bw_method=0.05, color='magenta', fill=True, label='HPC')

plt.title(str('kde peak times'))
plt.xlim([50,130])
plt.xticks(ticks=[60,80,100,120], labels=[0.6,0.8,1.0,1.2])
# plt.yticks(ticks=[0,0.02,0.04])
plt.xlabel('Time (s)')
plt.legend()

# Change the end of this next line with a new file name!!
to_plot = 0
if to_plot == 1:
    fig_title = str('V1HPC_oscillators_hist.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)
    
plt.show()

---

# Now looking at the frequency of these units

In [ ]:
v1_array = np.array(V1_rg.pivot('cuid', 'times', 'zscore'))
print(v1_array.shape) #this is [units x times] and the values are above

hpc_array = np.array(HPC_rg.pivot('cuid', 'times', 'zscore'))
print(hpc_array.shape) #this is [units x times] and the values are above

In [24]:
durationA = []
indA = []
for unit in range(v1_array.shape[0]):
    yy = v1_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    durationA.append(dur)
    indA.append(peak_ind)

durationB = []
indB = []
for unit in range(hpc_array.shape[0]):
    yy = hpc_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    durationB.append(dur)
    indB.append(peak_ind)
    
indA_array = np.concatenate(indA, axis=0)
indB_array = np.concatenate(indB, axis=0)

In [25]:
mean_freqA = [1/np.diff(i)*100+1 for i in indA]
mean_freqB = [1/np.diff(i)*100+1 for i in indB]

In [ ]:
# this is a dataframe of all cycle frequencies
allA_freq = np.concatenate(mean_freqA)
allB_freq = np.concatenate(mean_freqB)

all_freqA_df = pd.DataFrame(tuple(zip(allA_freq,(['v1']*len(allA_freq)))),columns=['freq','group'])
all_freqB_df = pd.DataFrame(tuple(zip(allB_freq,(['hpc']*len(allB_freq)))),columns=['freq','group'])
big_freq_df = pd.concat([all_freqA_df,all_freqB_df], ignore_index=True)

big_freq_df.reset_index(inplace=True)

big_freq_df.head()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 8.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.scatterplot(x='index', y='freq', data=big_freq_df, hue='group', linewidth=0.5)
plt.xlabel('unit index')
plt.ylabel('Frequency (Hz)')

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
# sns.histplot(data=big_freq_df, x="freq", hue="group")
sns.kdeplot(data=big_freq_df, x="freq", hue="group",
            cumulative=False, common_norm=False, bw_method=0.7, fill=True, legend=False)

plt.title('V1 - HPC')
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,13])
plt.yticks([0,0.1,0.2,0.3])
plt.ylim([0,0.35])

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\frequency'
if to_plot == 1:
    fig_title = str('V1HPC_all_peaks_line.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.kdeplot(data=big_freq_df, x="freq", hue="group",
            cumulative=True, common_norm=False,
            bw_method=0.7,legend=False)

plt.title('V1 - HPC')
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,11])

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\frequency'
if to_plot == 1:
    fig_title = str('V1HPC_all_peaks_cumsum.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 2.0 # x dimension
fig_size[1] = 5.0 # y dimension

ax1=plt.subplot(111)
sns.barplot(data=big_freq_df, x='group', y='freq', order=["v1","hpc"], ci=68)

plt.ylabel('Frequency (Hz)')
plt.xlabel('')
ax1.set_xticklabels(['V1','HPC'])
plt.yticks(ticks=[0,2,4,6])
# plt.ylim([0,6.5])
plt.title('V1-HPC')

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\frequency'
if to_plot == 1:
    fig_title = str('V1HPC_all_peaks_bar.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

In [ ]:
# upper-cycles frequency
result = sstat.ks_2samp(big_freq_df[big_freq_df['group']=='v1'].freq, 
                        big_freq_df[big_freq_df['group']=='hpc'].freq)
print('Oscillation Frequency (all cycles)')
print('V1: {0} --- HPC: {1}'.format(big_freq_df[big_freq_df['group']=='v1'].freq.nunique(),
                                    big_freq_df[big_freq_df['group']=='hpc'].freq.nunique()))
print(result)

# upper-cycles frequency
print('Oscillation Frequency (all cycles)')
print('Mean of V1: {0}'.format(np.mean(big_freq_df[big_freq_df['group']=='v1'].freq)))
print('Mean of HPC: {0}'.format(np.mean(big_freq_df[big_freq_df['group']=='hpc'].freq)))
result = sstat.ttest_ind(big_freq_df[big_freq_df['group']=='v1'].freq, 
                         big_freq_df[big_freq_df['group']=='hpc'].freq)
print(result)

In [ ]:
# this is a dataframe of specific cycle frequencies
freqA_df = pd.DataFrame(tuple(zip(mean_freqA,(['v1']*len(mean_freqA)))),columns=['freq','group'])
freqB_df = pd.DataFrame(tuple(zip(mean_freqB,(['hpc']*len(mean_freqB)))),columns=['freq','group'])
all_freq_df=pd.concat([freqA_df,freqB_df])

all_freq_df['up_cyc'] = all_freq_df.freq.str[1] #this is the 2-3 peak cycle
# all_freq_df['up_cyc2'] = all_freq_df.freq.str[2] #this is the 3-4 peak cycle

all_freq_df.head()

In [ ]:
# removes the NaN values from the above dataframe
all_freq_df=all_freq_df[~all_freq_df.up_cyc.isna()]
# all_freq_df=all_freq_df[~all_freq_df.up_cyc2.isna()]
all_freq_df.shape

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
# sns.histplot(data=all_freq_df, x="up_cyc", hue="group")
sns.kdeplot(data=all_freq_df, x="up_cyc", hue="group",
            cumulative=False, common_norm=False, bw_method=0.7, fill=True, legend=False)

plt.title('V1 - HPC')
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,13])
plt.yticks([0,0.1,0.2,0.3])
plt.ylim([0,0.35])

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\frequency'
if to_plot == 1:
    fig_title = str('V1HPC_upper_peaks_line.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.kdeplot(data=all_freq_df, x="up_cyc", hue="group",
            cumulative=True, common_norm=False,
            bw_method=0.7,legend=False)

plt.title('V1 - HPC')
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,11])

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\frequency'
if to_plot == 1:
    fig_title = str('V1HPC_upper_peaks_cumsum.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 2.0 # x dimension
fig_size[1] = 5.0 # y dimension

ax1=plt.subplot(111)
sns.barplot(data=all_freq_df, x='group', y='up_cyc', order=["v1","hpc"], ci=68)

plt.ylabel('Frequency (Hz)')
plt.xlabel('')
ax1.set_xticklabels(['V1','HPC'])
plt.yticks(ticks=[0,2,4,6])
# plt.ylim([0,6.5])
plt.title('V1-HPC')

# Change the end of this next line with a new file name!!
to_plot = 0
pathway = r'C:\Users\AChub_Lab\Desktop\zimmer94\Data_Analysis\Multi_brain_regions\saved_figs\frequency'
if to_plot == 1:
    fig_title = str('V1HPC_upper_peaks_bar.pdf')
    out_title = pathway+'\\'+fig_title
    plt.savefig(out_title, transparent=True)

plt.show()

In [ ]:
# upper-cycles frequency
result = sstat.ks_2samp(all_freq_df[all_freq_df['group']=='v1'].up_cyc, 
                        all_freq_df[all_freq_df['group']=='hpc'].up_cyc)

print('Oscillation Frequency (upper cycles)')
print('V1: {0} --- HPC: {1}'.format(all_freq_df[all_freq_df['group']=='v1'].up_cyc.nunique(),
                                    all_freq_df[all_freq_df['group']=='hpc'].up_cyc.nunique()))

print(result)

In [ ]:
# upper-cycles frequency
print('Oscillation Frequency (upper cycles)')
print('Mean of V1: {0}'.format(np.mean(all_freq_df[all_freq_df['group']=='v1'].up_cyc)))
print('Mean of HPC: {0}'.format(np.mean(all_freq_df[all_freq_df['group']=='hpc'].up_cyc)))

result = sstat.ttest_ind(all_freq_df[all_freq_df['group']=='v1'].up_cyc, 
                         all_freq_df[all_freq_df['group']=='hpc'].up_cyc)

print(result)

---

---

---